### Context

I've just opened a "PEA" account (Plan d'épargne en Actions) in my bank which is a tax-advantaged savings and investment plan designed to encourage long-term investment in the european stock market. So, I have started this project in order to build a profitable stocks portfolio for my personal interest.

I hope this project helps you !

### Librairies needed

In [1]:
#API Financial Data
import yfinance as yf

#For Data Treatment
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd
import numpy as np
import itertools

#Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#For Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Optimization
import cvxopt as opt
from cvxopt import blas, solvers

#Others
import warnings

warnings.filterwarnings('ignore')
sns.set_theme()

/Users/ayoub/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# I - Data retrievement

In [2]:
pd.ExcelFile("tickers.xlsx").sheet_names

['Stock', 'ETF', 'Future', 'Index', 'Mutual Fund', 'Currency', 'About']

#### Target Asset Parameter

In [3]:
targeted_asset = "Stock"

In [ ]:
european_stocks = pd.read_excel("tickers.xlsx", sheet_name = targeted_asset)
european_stocks = european_stocks.rename(columns=european_stocks.iloc[2]).drop(index=[0, 1, 2]) 
columns_to_keep = [column for column in european_stocks.columns if type(column) == str] 
european_stocks = european_stocks[columns_to_keep]
european_stocks.head(10)

In [ ]:
#Country
print(list(set(european_stocks["Country"].to_list())))

In [ ]:
#Exchange
print(list(set(european_stocks["Exchange"].to_list())))

Index Tickers

- "^FCHI"(CAC 40 --> France)
- "^GDAXI" (DAX --> Germany)
- "^FTSE" (FTSE --> GB)
- "^GSPC" (S&P --> US)

#### Targeted Market Parameter

In [ ]:
countries = ["France"]
exchange = ["PAR"]
index_ticker = "^GSPC"
start_date = "2018-06-20"
end_date = "2023-06-20"
period = (dt.strptime(end_date, "%Y-%m-%d")-dt.strptime(start_date, "%Y-%m-%d")).days

## Stocks Data

In [ ]:
european_stocks_s= european_stocks[european_stocks["Country"].isin(countries)].dropna(subset=["Category Name"])
if len(exchange) != 0:
    european_stocks_s = european_stocks[european_stocks["Exchange"].isin(exchange)].dropna(subset=["Category Name"])
european_stocks_s

#### Dataframe compiling all daily returns from all tickers in the european_stocks_tickers datframe

In [ ]:
df_prices = pd.DataFrame()
i = 0
for ticker, name, category in zip(european_stocks_s["Ticker"], european_stocks_s["Name"], european_stocks_s["Category Name"]):
    data = yf.Ticker(ticker).history(start=start_date, end=end_date, period="1d")
    if i == 0:
        x = len(data)
        i += 1
    if data["Volume"].sum() > period*(10**5) and data["Close"].sum() > period*10 and len(data) == x:
        df_prices["{}/{}/{}".format(name, ticker, category)] = data["Close"]

In [ ]:
df_prices.index = df_prices.index.strftime('%Y-%m-%d')
df_prices

In [ ]:
df_returns = 100*df_prices.pct_change().dropna()
df_returns

## Risk free Rate Data ("OAT" French Bunds)

In [ ]:
risk_free_rate_data = pd.read_csv("oat_rates.csv", delimiter=";")
risk_free_rate_data = risk_free_rate_data.set_index(risk_free_rate_data["Titre :"]).drop(columns=["Titre :"])
risk_free_rate_data.index.name = "Date"
risk_free_rate_data = risk_free_rate_data.drop(index=["Code série :", "Unité :", "Magnitude :", "Méthode d'observation :", "Source :"])
risk_free_rate_data.index = pd.to_datetime(risk_free_rate_data.index, dayfirst=True)
risk_free_rate_data = risk_free_rate_data[risk_free_rate_data[risk_free_rate_data.columns] != "-"].loc[start_date:end_date].dropna()
risk_free_rate_data

In [ ]:
risk_free_rate_data = risk_free_rate_data[["Taux indicatifs des bons du trésor à 1 mois","Taux indicatifs des bons du trésor à 3 mois","Taux indicatifs des bons du trésor à 6 mois","Taux indicatifs des bons du trésor à 9 mois","Taux indicatifs des bons du trésor à 12 mois","taux indicatifs des OAT 2 ans","taux indicatifs des OAT 5 ans","Emprunt Phare 10 ans","Taux de l'OAT à 30 ans - France"]]
risk_free_rate_data.loc["2023-04-25"]

#### Example of a raw rate curve

In [ ]:
plt.figure(figsize=(14,5))
real_buckets = [round(1/12, 3), round(3/12, 3), round(6/12, 3), round(9/12, 3), 1.0, 2.0, 5.0, 10.0, 30.0]
rates_test = [float(elem.replace(",", ".")) for elem in risk_free_rate_data.loc["2022-03-22"]]
plt.plot(real_buckets, rates_test, color="red")
plt.xlabel("Maturity")
plt.ylabel("Interest Rates")
plt.show()

In [ ]:
#Missing bucket (~weekly period) and construction of daily OAT curve by a linear interpolation
target_buckets = [round(i/52, 3) for i in range((30*52)+1)]
rates_curves = []
for date in risk_free_rate_data.index:
    rates_data = risk_free_rate_data.loc[date].to_list()
    rates_curves_daily = [float(rates_data[0].replace(",", "."))]
    moving_real_buckets = real_buckets.copy()
    for target_bucket in target_buckets:
        if round(target_bucket, 3) in real_buckets:
            rates_curves_daily.append(float(rates_data[real_buckets.index(round(target_bucket, 3))].replace(",", ".")))
        else:
            for real_bucket in moving_real_buckets:
                if target_bucket < real_bucket:
                    x_b, y_b = real_bucket, float(rates_data[real_buckets.index(real_bucket)].replace(",", "."))
                    x_a, y_a = real_buckets[real_buckets.index(real_bucket)-1], float(rates_data[real_buckets.index(real_bucket)-1].replace(",", "."))
                    inter_rate = ((x_b-target_bucket)/(x_b-x_a))*y_a + ((target_bucket-x_a)/(x_b-x_a))*y_b
                    rates_curves_daily.append(inter_rate)
                    moving_real_buckets.insert(moving_real_buckets.index(real_bucket), target_bucket)
                    break
    rates_curves.append([date] + rates_curves_daily[2:])

In [ ]:
col_names = ["Date"] + ["OAT {} year(s)".format(bucket) for bucket in target_buckets[1:]]
data_rates = pd.DataFrame(rates_curves)
data_rates.columns = col_names
data_rates = data_rates.set_index(data_rates["Date"]).drop(columns=["Date"])
data_rates.index = data_rates.index.strftime("%Y-%m-%d")
data_rates

#### Exemple of a rate curve after linear interpolations

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(target_buckets[1:], data_rates.loc["2022-03-22"], color="red")
plt.xlabel("Maturity")
plt.ylabel("Interest Rates")
plt.show()

In [ ]:
print(period/365)
for target in target_buckets:
    if target > (period/365) and (abs(target - (period/365)) < abs(target_buckets[target_buckets.index(target)-1]-(period/365))):
        index_t = target_buckets.index(target)
        break
    elif target > (period/365) and (abs(target - (period/365)) > abs(target_buckets[target_buckets.index(target)-1]-(period/365))):
        index_t = target_buckets.index(target)-1
        break
index_t

In [ ]:
data_rates.iloc[:, index_t]

## Final DataFrame

In [ ]:
available_dates = [date for date in df_returns.index if date in data_rates.index]
df_returns = df_returns[df_returns.index.isin(available_dates)]
df_returns.insert(len(df_returns.columns), "Rates", data_rates.iloc[:, index_t])
df_returns

# II - Individual Stocks Assessment

In [ ]:
df_r = df_returns.copy()

In [ ]:
#Stocks "Name/Yahoo_Symbol/Category"
L = [stock for stock in df_returns.columns if "/" in stock]
L

### Start with an example to illustrate the procedure

In [ ]:
#Choose your stock
stock = L[0]

In [ ]:
data = yf.Ticker(stock.split("/")[1]).history(start=start_date, end=end_date, period="1d")
data.index = data.index.strftime('%Y-%m-%d')
data

In [ ]:
#Global variable for ratios
return_period = np.divide(100*(np.array(data["Close"].diff().dropna()) + np.array(data["Dividends"][1:])), np.array(data["Close"].to_list()[:-1]))
av_return_period = return_period.mean() * 252
risk_free_rate = df_returns["Rates"].loc[data.index[1]]
stock_vol = round(return_period.std(), 2) * np.sqrt(252)

#### Sharpe Ratio

In [ ]:
sharpe_ratio = (av_return_period-risk_free_rate)/stock_vol
print("Ratio de Sharpe: ", sharpe_ratio)

#### Sortino Ratio

In [ ]:
neg_return = [return_ for return_ in return_period if return_ < 0]
if len(neg_return) > 1:
    stock_vol_sortino = np.std(neg_return) * np.sqrt(252)
    sortino_ratio = (av_return_period - risk_free_rate)/stock_vol_sortino
else:
    sortino_ratio = 0
print("Ratio de Sortino: ", sortino_ratio)

#### Benchmark Index

In [ ]:
start_date_index = dt.strptime(df_returns.index[0], "%Y-%m-%d") - timedelta(days=30)
end_date_index = dt.strptime(df_returns.index[0], "%Y-%m-%d") + timedelta(days=30)
index_ref = yf.Ticker(index_ticker).history(start=start_date_index, end=end_date, period="1d")
index_ref.index = index_ref.index.strftime("%Y-%m-%d")

In [ ]:
index_returns = pd.DataFrame(np.divide(100*(np.array(index_ref["Close"].diff().dropna()) + np.array(index_ref["Dividends"][1:])), np.array(index_ref["Close"].to_list()[:-1])), \
                             index= index_ref.index[1:], columns=["Index Returns"])

In [ ]:
if "Index Returns" in df_r.columns:  
    del df_r["Index Returns"]
df_r.insert(len(df_returns.columns)-1, "Index Returns", index_returns[index_returns.index.isin(df_r.index)], True)

In [ ]:
#Who outperforms ? 
if sum(np.array(df_r[stock])-np.array(df_r["Index Returns"])) > 0:
    print("L'action de {} a surperformé l'indice !!".format(stock.split("/")[0]))
else:
    print("L'indice a surperformé l'action de {}".format(stock.split("/")[0]))

In [ ]:
df_r = df_r.dropna()
df_r

#### Treynor Ratio

##### Deals with outliers by the IQR method

In [ ]:
#Deals with outliers BY the IQR method
outliers_index_by_stock = []
i = 0
df_o = df_r.copy()
for col in df_o.columns[:-2]:
    Q1 = df_o[col].quantile(0.25)
    Q3 = df_o[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1 - 1.5*IQR
    upper_whisker = Q3 + 1.5*IQR
    outliers_index_by_stock.append([col, list(df_o[(df_o[col] >= upper_whisker) | (df_o[col] <= lower_whisker)][col].index)])

In [ ]:
max_len_outliers = len(max(outliers_index_by_stock, key=lambda x: len(x[1]))[1])
max_len_outliers

In [ ]:
for outlier in outliers_index_by_stock:
    n = len(outlier[1])
    if n < max_len_outliers:
        outlier[1] = outlier[1] + [np.nan]*(max_len_outliers-n)

In [ ]:
df_outliers = pd.DataFrame()
i = 0 
for outlier in outliers_index_by_stock:
    df_outliers.insert(i, outlier[0], outlier[1])
df_outliers

##### Linear Regression (Index Returns, Stock Returns)

In [ ]:
reg_dict = {}
for col in df_outliers.columns:
    df_reg = pd.DataFrame()
    l_o = [elem for elem in df_outliers[col] if type(elem) == str]
    df_reg.insert(0, col + " Returns", df_r[~df_r.index.isin(l_o)][col])
    df_reg.insert(1, "Index Returns", df_r[~df_r.index.isin(l_o)]["Index Returns"])
    df_reg.insert(2, "Rates", df_r[~df_r.index.isin(l_o)]["Rates"])
    reg_dict[col] = df_reg

In [ ]:
kik = reg_dict[stock].columns
reg_dict[stock]

In [ ]:
plt.figure(figsize=(15, 4))
plt.scatter(reg_dict[stock]["Index Returns"], reg_dict[stock]["{} Returns".format(stock)], color="red")
plt.xlabel("Index Returns")
plt.ylabel("Stock Returns")
plt.show()

In [ ]:
#Scikit learn method
model = LinearRegression()
X, y = np.array(reg_dict[stock]["Index Returns"]), np.array(reg_dict[stock]["{} Returns".format(stock)])
X = X.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
reg = model.fit(X_train, y_train)

In [ ]:
r_sq = reg.score(X, y)
print('coefficient of determination:', r_sq)

print('Intercept:', reg.intercept_)

print('Slope:', reg.coef_) 

y_pred = reg.predict(X)

In [ ]:
plt.figure(figsize=(15, 4))
plt.scatter(reg_dict[stock]["Index Returns"], reg_dict[stock]["{} Returns".format(stock)], color="red")
plt.plot(reg_dict[stock]["Index Returns"], y_pred, color="blue")
plt.xlabel("Index Returns")
plt.ylabel("Stock Returns")
plt.show()

In [ ]:
beta = reg.coef_[0]
print("Treynor Ratio: ", ((av_return_period/252)-risk_free_rate)/beta)

### Now Let's apply the procedure to all stocks !

In [ ]:
#Reminder
print(f"Period: {start_date} ==> {end_date}")

In [ ]:
df_stocks_perf = pd.DataFrame(index=["Prix initial-->Prix finale", "Total Stock Return (%)", "Annual Stock Return (%)" ,"Volatility (%)", "Risk Free Rate (%)" ,"Sharpe Ratio", "Sortino Ratio", "Treynor Ratio" , "CAC 40 Correlation"])
for stock in df_r.columns[:-2]:
    print(stock)
    data = yf.Ticker(stock.split("/")[1]).history(start=start_date,end=end_date ,period="1d") 
    data.index = data.index.strftime('%Y-%m-%d')
    dates = data.index
    #Price Growth
    prices = "  ==> ".join([str(round(data["Close"].loc[dates[0]], 2)), str(round(data["Close"].loc[dates[-1]], 2))])
    growth = ((data["Close"].loc[dates[-1]]-data["Close"].loc[dates[0]])/data["Close"].loc[dates[0]])
    period = ((dt.strptime(dates[-1], "%Y-%m-%d")-dt.strptime(dates[0], "%Y-%m-%d")).days)/365
    ann_growth = ((1+growth)**(1/period))-1
    growth = round(growth*100, 2)
    ann_growth = round(ann_growth*100, 2)
    #Global variable for ratios
    return_period = np.divide(100*(np.array(data["Close"].diff().dropna()) + np.array(data["Dividends"][1:])), np.array(data["Close"].to_list()[:-1]))
    av_return_period = return_period.mean()*252
    risk_free_rate = df_returns["Rates"].loc[df_returns.index[0]]
    period_volatility_percentage = round(return_period.std()*np.sqrt(252), 2)
    #Sharpe Ratio
    sharpe_ratio = (av_return_period-risk_free_rate)/period_volatility_percentage
    #Sortino Ratio
    neg_return = [return_ for return_ in return_period if return_ < 0]
    if len(neg_return) > 1:
        stock_vol_sortino = np.std(neg_return)*np.sqrt(252)
        sortino_ratio = (av_return_period - risk_free_rate)/stock_vol_sortino
    else:
        sortino_ratio = 0
    #Treynor Ratio
    model = LinearRegression()
    X, y = np.array(reg_dict[stock]["Index Returns"]), np.array(reg_dict[stock]["{} Returns".format(stock)])
    X = X.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    reg = model.fit(X_test, y_test)
    beta = reg.coef_[0]
    treynor_ratio = ((av_return_period/252)-risk_free_rate)/beta
    #Add values in df
    df_stocks_perf[stock] = [prices, growth, ann_growth, period_volatility_percentage, round(risk_free_rate, 4), sharpe_ratio, sortino_ratio, treynor_ratio, reg.score(X,y)]

In [ ]:
df_stocks_perf

In [ ]:
#Checking result for LVMH
df_stocks_perf[L[0]]

In [ ]:
df_most = df_stocks_perf[df_stocks_perf.loc["Total Stock Return (%)":] >= 0]
df_most = df_most.dropna(thresh=len(df_most) - 2, axis=1)

In [ ]:
#Top Stock Return
df_most_return = df_most.dropna(thresh=len(df_most) - 2, axis=1).sort_values("Total Stock Return (%)", axis=1, ascending=False)
stock_name_r = [elem.split("/")[0] for elem in df_most_return]
list(zip(stock_name_r, df_most_return.loc["Total Stock Return (%)"]))

In [ ]:
#Top Volatility
df_most_vol = df_most.dropna(thresh=len(df_most) - 2, axis=1).sort_values("Volatility (%)", axis=1, ascending=False)
stock_name_vol = [elem.split("/")[0] for elem in df_most_vol]
list(zip(stock_name_vol, df_most_vol.loc["Volatility (%)"]))

In [ ]:
#Top Sharpe Ratio
df_most_sharpe = df_most.dropna(thresh=len(df_most) - 2, axis=1).sort_values("Sharpe Ratio", axis=1, ascending=False)
stock_name_sh = [elem.split("/")[0] for elem in df_most_sharpe]
list(zip(stock_name_sh, df_most_sharpe.loc["Sharpe Ratio"]))

In [ ]:
#Top Sortino Ratio
df_most_sortino = df_most.dropna(thresh=len(df_most) - 2, axis=1).sort_values("Sortino Ratio", axis=1, ascending=False)
stock_name_so = [elem.split("/")[0] for elem in df_most_sortino]
list(zip(stock_name_so, df_most_sortino.loc["Sortino Ratio"]))

In [ ]:
#Top Treynor Ratio
df_most_treynor = df_most.dropna(thresh=len(df_most) - 2, axis=1).sort_values("Treynor Ratio", axis=1, ascending=False)
stock_name_tr = [elem.split("/")[0] for elem in df_most_treynor]
list(zip(stock_name_tr, df_most_treynor.loc["Treynor Ratio"]))

In [ ]:
#Top Index/Stock coeff of determination
df_most_info = df_most.dropna(thresh=len(df_most) - 2, axis=1).sort_values("CAC 40 Correlation", axis=1, ascending=False)
stock_name_info = [elem.split("/")[0] for elem in df_most_info]
list(zip(stock_name_info, df_most_info.loc["CAC 40 Correlation"]))

# III- Portfolio Construction

### Goals
- Diverisified portfolio with 4 stocks
- At least a global return of 12%
- Sharpe, Sortino and Treynor ratios upper than 1

### Constraints
- At least 5% of ponderation for each stock and max 30%
- Correlation limit between stocks of 0.3
- Volatility limit of 18%

### Methods
- Monte Carlo
- Markowitz Model

In [ ]:
######## Global Variables ########
number_of_stocks = 4

########### Monte Carlo ###########
#Constraints
expected_annual_return = 0.20
min_ratios = 0.5
pond_max = 0.30
pond_min = 0.05
corr_limit = 0.35
vol_max = 0.18
#Simulation
number_of_simulations_by_diversified_pf = 10000

#Markowitz Model
number_of_random_portfolios = 1500

### Dataframes

In [ ]:
#Daily Stocks Returns, Daily Index CAC 40 returns, Daily Free Risk Rates data
df_returns_p = df_r.loc[start_date:end_date].copy()
df_returns_p

In [ ]:
#Stocks Performance
df_perf_p = df_stocks_perf[df_most_return.columns].copy()
df_perf_p

In [ ]:
#Correlation matrix
correlation_matrix = df_returns_p[df_most_return.columns].corr()
correlation_matrix

### Monte Carlo Simulation

In [ ]:
def weighting_list(n):
    k = np.random.rand(n)
    return k / sum(k)

In [ ]:
#All combinations possible without repeated elements
stocks = correlation_matrix.columns[:30]
combinations = list(itertools.combinations(stocks, number_of_stocks))

len(combinations)

In [ ]:
less_correlated_stocks = []
for combination in combinations:
    combination = list(combination)
    sub_matrix = correlation_matrix.loc[combination, combination]
    if (sub_matrix.values[~pd.DataFrame(np.eye(len(sub_matrix), dtype=bool)).values] < corr_limit).all():
        less_correlated_stocks.append(combination)
print("Number of Diversified Combinations:", len(less_correlated_stocks))

In [ ]:
#Best diversified portfolios
portfolios = []
index_re_col = df_returns_p["Index Returns"]
rates_re_col = df_returns_p["Rates"]
dates = df_returns_p.index
num = -1
total_num = 0
for combination in less_correlated_stocks:
    if num == -1:
        print("The Process Started...\n")
        num = 0
    else:
        print(less_correlated_stocks[less_correlated_stocks.index(combination)-1])
        print("Number of allocations:", num)
        print("\n", "-"*100, "\n")
        total_num += num
        num = 0
    combination = list(combination)
    for _ in range(number_of_simulations_by_diversified_pf):
        sub_df = pd.DataFrame()
        sub_matrix = correlation_matrix.loc[combination, combination]
        if (sub_matrix.values[~pd.DataFrame(np.eye(len(sub_matrix), dtype=bool)).values] < corr_limit).all():
            weights = weighting_list(number_of_stocks)
            if (max(weights) > pond_max and min(weights) < pond_min) or int(sum(weights)) == 0 :
                continue
            i = 0
            start_return, end_return, portfolio_return = 0, 0, 0
            for stock, weight in zip(combination, weights):
                try:
                    i += 1
                    portfolio_return += weight*df_returns_p[stock]
                    start_return += weight*df_prices[stock].loc[dates[0]:dates[-1]].to_list()[0]
                    end_return += weight*df_prices[stock].loc[dates[0]:dates[-1]].to_list()[-1]
                except:
                    break

            if i != number_of_stocks:
                continue

            sub_df.insert(0, "Portfolio Returns", portfolio_return, True)
            sub_df.index = df_returns_p.index
            sub_df.insert(1, "Index Returns", index_re_col, True)
            sub_df.insert(2, "Rates", rates_re_col, True)

            #Price Growth
            growth = ((end_return-start_return)/start_return)
            period = ((dt.strptime(dates[-1], "%Y-%m-%d")-dt.strptime(dates[0], "%Y-%m-%d")).days)/365
            ann_growth = ((1+growth)**(1/period))-1
            if ann_growth < expected_annual_return:
                continue
            ann_growth = round(ann_growth*100, 2)
            growth = round(growth*100, 2)

            #Global variable for ratios
            av_return_period = round(portfolio_return.mean()*252, 3)
            risk_free_rate = round(df_returns_p["Rates"].loc[dates[0]], 3)
            period_volatility_percentage = round(portfolio_return.std()* np.sqrt(252),3)
            if period_volatility_percentage > vol_max*100:
                continue

            #Sharpe Ratio
            sharpe_ratio = (av_return_period-risk_free_rate)/period_volatility_percentage
            if sharpe_ratio < min_ratios:
                continue

            #Sortino Ratio
            neg_return = [return_ for return_ in portfolio_return if return_ < 0]
            if len(neg_return) > 1:
                stock_vol_sortino = np.std(neg_return) * np.sqrt(252)
                sortino_ratio = (av_return_period - risk_free_rate)/stock_vol_sortino
            else:
                sortino_ratio = 0

            #Treynor Ratio
            model = LinearRegression()
            X, y = np.array(index_re_col), portfolio_return
            X = X.reshape(-1, 1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
            reg = model.fit(X_test, y_test)
            beta = reg.coef_[0]
            treynor_ratio = (av_return_period-risk_free_rate)/beta

            num += 1
            combo = [elem.split("/")[0] for elem in combination]
            best_correlation_combination = {
                "Combination": " + ".join(list(map(lambda x: f"{x[0]}*{x[1]}", zip(weights, combo)))),
                "Dataframe": sub_df,
                "Total Growth": f"{growth}%",
                "Annual Growth": f"{ann_growth}%",
                "Expected Return": f"{av_return_period}%",
                "Volatility": f"{period_volatility_percentage}%",
                "Risk Free Rate": f"{risk_free_rate}%",
                "Sharpe ratio": round(sharpe_ratio, 3),
                "Sortino ratio": round(sortino_ratio, 3),
                "Treynor ratio": round(treynor_ratio, 3),
                "CAC 40 corr": round(reg.score(X,y), 3),
                "Beta": round(beta, 3),
                "Alpha": round(reg.intercept_, 3)
            }
            portfolios.append(best_correlation_combination)
            
        
print("Number of diversified Portfolios:", total_num, "\n")

print("-------------------- Report --------------------")
for key in portfolios[0].keys():
    how_ = True
    if key in ["Volatility", "Beta"]:
        how_ = False
    elif key in ["Combination", "Risk Free Rate", "Dataframe"]:
        continue
    print(f"\n************* By {key} *************")
    best_correlation_combinations = sorted(portfolios, key=lambda x: float(x[f"{key}"][:-1]) if type(x[f"{key}"]) == str else x[f"{key}"], reverse=how_)
    print("Combination:", best_correlation_combinations[0]["Combination"])
    print("Total Growth:", best_correlation_combinations[0]["Total Growth"])
    print("Annual Growth:", best_correlation_combinations[0]["Annual Growth"])
    print("Expected Return:", best_correlation_combinations[0]["Expected Return"])
    print("Volatility:", best_correlation_combinations[0]["Volatility"])
    print("Risk Free Rate:", best_correlation_combinations[0]["Risk Free Rate"])
    print("Sharpe ratio:", best_correlation_combinations[0]["Sharpe ratio"])
    print("Sortino ratio:", best_correlation_combinations[0]["Sortino ratio"])
    print("Treynor ratio:", best_correlation_combinations[0]["Treynor ratio"])
    print("CAC 40 coef of deter. :", best_correlation_combinations[0]["CAC 40 corr"])
    print("Beta: ", best_correlation_combinations[0]["Beta"])
    print("Alpha: ", best_correlation_combinations[0]["Alpha"])

## Markowitz Portfolio Theory

In [ ]:
r_portfolio = df_returns_p[['LVMH Moët Hennessy Louis Vuitton S.E./MC.PA/Jewelry Stores', 'Sanofi/SAN.PA/Drug Manufacturers - Major', 'Eurofins Scientific SE/ERF.PA/Medical Laboratories & Research', 'Thales S.A./HO.PA/Aerospace/Defense - Major Diversified']]
r_portfolio

In [ ]:
def sim_rand_portfolios(returns):

    mean_matrix = np.asmatrix(np.mean(returns, axis=0))
    weight_matrix = np.asmatrix(weighting_list(returns.T.shape[0]))
    cov_matrix = np.asmatrix(np.cov(returns.T))
    
    
    expected_return = weight_matrix * mean_matrix.T * 252
    volatility = np.sqrt(weight_matrix * cov_matrix * weight_matrix.T) * np.sqrt(252)
    
    # This recursion reduces outliers to keep plots pretty
    if volatility > 90:
        return sim_rand_portfolios(returns)
    return expected_return, volatility

means_return, vols = np.column_stack([
    sim_rand_portfolios(r_portfolio) 
    for _ in range(number_of_random_portfolios)
])

In [ ]:
fig = plt.figure(figsize=(14, 7))
plt.plot(vols, means_return, 'o', markersize=5)
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.title('Mean and standard deviation of returns of randomly generated portfolios')
plt.show()

In [ ]:
def sim_optimization_portfolio(returns):
    returns = returns.T
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 10
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    G = -opt.matrix(np.eye(n))  
    h = opt.matrix(0.1, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]


    returns = [252*blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x))*np.sqrt(252) for x in portfolios]
    

    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return list(wt), returns, risks

weights, returns, risks = sim_optimization_portfolio(r_portfolio)

fig = plt.figure(figsize=(14,5))
plt.plot(vols, means_return, 'o')
plt.ylabel('Expected Return')
plt.xlabel('Volatility')
plt.plot(risks, returns, 'y-o')

### Comparison

In [ ]:
mpt_results = list(zip(weights, r_portfolio.columns))
mpt_results

In [ ]:
monte_carlo_results = sorted(portfolios, key=lambda x: float(x["Sharpe ratio"][:-1]) if type(x["Sharpe ratio"]) == str else x["Sharpe ratio"])[0]["Combination"]
monte_carlo_results = [[float(elem.split('*')[0]), elem.split('*')[1]] for elem in monte_carlo_results.split("+")]
monte_carlo_results

In [ ]:
companies = [elem[1].split()[0] for elem in monte_carlo_results]
width = 3
x = np.array(list(range(1, len(companies)*10, 10)))
fig, ax = plt.subplots()
ax.bar(x - width/2, [elem[0] for elem in monte_carlo_results], width, color="blue", label="Monte Carlo Results")
ax.bar(x + width/2, [elem[0] for elem in mpt_results], width, color="green", label="MPT Results")
ax.set_xticks(x)
ax.set_xticklabels(companies)
ax.set_title("Weights Comparaison")
ax.legend()
plt.show()

In [ ]:
import matplotlib.dates as mdates

p_sh = sorted(portfolios, key=lambda x: float(x["Sharpe ratio"][:-1]) if type(x["Sharpe ratio"]) == str else x["Sharpe ratio"])[0]["Dataframe"]
returns_monte_carlo, index_returns = p_sh["Portfolio Returns"]/100, p_sh["Index Returns"]/100
capital_mpt, capital_monte, capital_index = 1000, 1000, 1000

returns_mpt = 0
ws_mpt = [elem[0] for elem in mpt_results] #Optimized Allocation
#ws_mpt = [0, 1, 0, 0] #Manual allocation
companies = [elem[1] for elem in mpt_results]
for w_mpt, stock in zip(ws_mpt, companies):
    returns_mpt += w_mpt/100*r_portfolio[stock]

r_monte_carlo = (1+returns_monte_carlo).to_list()
r_mpt = (1+returns_mpt).to_list()
r_index = (1+index_returns).to_list()

y_1, y_2, y_3 = [], [], []
for i in range(len(returns_monte_carlo)):
    res_mpt = capital_mpt * r_mpt[i]
    res_mont = capital_monte * r_monte_carlo[i]
    res_index = capital_index * r_index[i]
    y_1.append(res_mont)
    y_2.append(res_mpt)
    y_3.append(res_index)
    capital_mpt, capital_monte, capital_index = res_mpt, res_mont, res_index

dates = returns_monte_carlo.index
plt.figure(figsize=(20, 6))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=90))
plt.plot(dates, y_1, label="Monte Carlo Returns", color="blue")
plt.plot(dates, y_2, label="MPT Returns", color="green")
plt.plot(dates, y_3, label="Benchmark Index Returns", color="orange")
plt.legend()
plt.show()